In [1]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
from funcoes import CalculosRiscoPort

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
#Chama as classes do databases
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()

In [3]:
##Informações necessárias para rodar a otimização das carteiras- gerar as movimentações

#Tabela com as informações das SC de gestão offshore (nomes, responsáveis e perfil de cada cliente)
po_cadastro = bawm.po_cadastro_all().drop(columns=['CodigoProduto'])

#Puxa da PO_Cadastro os responsaveis por cada supercarteira de gestão (DPMS)

#Lista com os usuários e emails de toda Julius Baer Brazil.
emails_usuarios = crm.lista_usuarios_u()

#Verificar os responsaveis por cada sc de gestão com base no mapa de ativos
responsaveis = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore\Mapa de Ativos/Mapa JBFO_v2.xlsx',sheet_name='Mapa_DB')
responsaveis = responsaveis[(responsaveis['Origem']=='Offshore')& (responsaveis['Tipo de Contrato']!='Distrato em Andamento')][['Conta','SC/ContaMov','RM','Co-RM','Controller','Controller Backup']]

#Altera os usuários u para letra minúscula para padronização (será usado no envio de e-mail)
responsaveis['RM'] =responsaveis['RM'].str.lower()
responsaveis['Co-RM'] =responsaveis['Co-RM'].str.lower()
responsaveis['Controller'] = responsaveis['Controller'].str.lower()
responsaveis['Controller Backup'] = responsaveis['Controller Backup'].str.lower()
#responsaveis['DeputyDPMUsuarioU'] = responsaveis['DeputyDPMUsuarioU'].str.lower()


In [4]:
# Última data do mes anterior para rodar a query do extrato
data = bds.banco.hoje()
data = datetime.datetime.date(data)

#Traz a alocação ideal por perfil de portfolio
alocacao_por_sc = crm.alocacao_ideial_off()

#Traz os fundos da JBFO (prioridade na aplicação das movimentações)
fundos_jbfo = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')
fundos_jbfo = fundos_jbfo[['NomeDoProduto','CodigoProduto','Classificação']]
fundos_jbfo['Fundo']=fundos_jbfo['NomeDoProduto'].str.lower()

#de_paras de mandatos (PI anterior para a atual - equivalência)
mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_mandato')
mandatos = mandatos[['Anterior','Novo']]
#mandatos = dict(mandatos.values)
    
#Traz os deparas de classificação gps para classes de asset Allocation.
dicio_classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacoes=dicio_classificacoes[['Classificação GPS','De Para - Asset Allocation']]
dicio_classificacoes = dict(dicio_classificacoes.values)

#Novos mandatos - Pontos Táticos
novos_mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Novos_mandatos')
novos_mandatos= novos_mandatos[['Portfolio','Classe','Tático']]

#Clientes que estão em distrato
distrato = crm.clientes_em_distrato_off()

#Trazer os produtos GPS e segregar o que é imobiliario (caso precise segregar o que é Reits / ILiquidos)
produtos_gps = crm.produtos_off()
produtos_gps =produtos_gps[['name','new_idmysql','new_tickerid','new_isin','new_idbds','new_idsistemaorigem','new_indgroupaxysidname']].rename(columns={'name':'Ativo','new_idmysql':'IdMysqlProduto','new_tickerid':'Ticker','new_isin':'ISIN','new_idsistemaorigem':'CodigoProduto'}).reset_index()
produtos_gps['ISIN'] = produtos_gps['ISIN'].str.lower()

#Retorna o nome dos veiculos que fazemos gestao (substituindo a classe JBFO)
dicio_veiculos = dict(pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Fundos_Casa')[['Classe','Veiculo Abreviado']].drop_duplicates().values)

In [5]:
#Funções usadas na otimização de carteiras

#Retira espaços inúteis
def arruma_espaco(x):
    while x[-1]==' ':
        x=x[:-1]
    return x    
    
#Formatar valor para envio de e-mail
def formatar(valor):
    int_part, dec_part = format(valor, ',.2f').split('.')
    return f"{int_part.replace(',', '.')},{dec_part}"

#Trazer pu do BDS (ainda não usado)
def trazer_pu_atual(id_serie):
    try:
        df = bds.serie_historico(idserie=id_serie, intervalo=1)
        if df.empty:
            df = bds.serie_historico(idserie=id_serie, intervalo=4)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)          
        else:
            df = bds.serie_historico(idserie=id_serie, intervalo=1)
            cota = df.iloc[-1,5]
            ultima_data = df.iloc[-1,3]
            ultima_data =datetime.datetime.date(ultima_data)
    except:
        cota=np.nan
        ultima_data = np.nan
    return cota, ultima_data    

#Função que identifica clientes inativos
def inativo(x):
    x = x.replace('(','').replace(')','')
    x = x.replace(' ','')
    x = x.strip()
    if 'inativ' in x.lower():
        valor='inativo'
    else:
        valor='ok'
    return valor   

In [6]:
#Consolida a posição atual versus a ideial
alocacao_por_sc = pd.pivot_table(data=alocacao_por_sc, values = 'new_percentual',columns=['new_classedeinvestimentoidname'],index=['new_name','new_conta_supercarteiraidname','new_politicadeinvestimentoidname']).reset_index()
alocacao_por_sc['new_name']=alocacao_por_sc['new_name'].str.lower()
alocacao_por_sc = pd.merge(left = alocacao_por_sc, right = po_cadastro,left_on ='new_conta_supercarteiraidname',right_on = 'NomeSuperCarteiraCRM',how='left')
alocacao_por_sc['inativo']=alocacao_por_sc['new_name'].apply(lambda x: inativo(x))
alocacao_por_sc=alocacao_por_sc[alocacao_por_sc['inativo']!='inativo']
alocacao_por_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname','Alt Crédito', 'Alt Multistr','Alt RV L/S', 'Alt Trad/Macro', 'Alt Valor Rel', 'Commodities','Imobiliário', 'Outros', 'P. Equity', 'RF CP', 'RF Glob', 'RF Infl','RF Livre -', 'RF Outros -', 'RV', 'RV Outros -','new_politicadeinvestimentoidname','Titularidade','VetoPowerGestaoTatica','DPM','OfficerUsuarioU','ControllerUsuarioU']].reset_index(drop=True)

In [7]:
##Traz a posição do último extrato gerado

#Retorna o último dia do mes
ultimo_extrato = data - relativedelta(days=data.day -1)
ultimo_extrato = ultimo_extrato + timedelta(days = -1)

#Gera a carteira da primeira sc da lista
teste = alocacao_por_sc.drop_duplicates()
sc = teste.iloc[0,0]
#sc = '@5AGL1'
df = extrato.posicao_extrato(sc,ultimo_extrato)

#Se o extrato não foi processado no último dia do mês, trazer o último disponível
if df.empty:
    ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
    ultimo_extrato = ultimo_extrato + timedelta(days = -1)
else:
    ultimo_extrato = ultimo_extrato

    
#Gerar a carteira de todas as sc de gestão - portfólios padrão
for i,(index,row) in enumerate(teste.iterrows()):
     if i >0 :
            ultimo_extrato = data
            ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
            ultimo_extrato = ultimo_extrato + timedelta(days = -1)            
            sc = row['new_name']
            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            if df1.empty:
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
            else:
                ultimo_extrato = data
                ultimo_extrato = ultimo_extrato - relativedelta(days=ultimo_extrato.day -1)
                ultimo_extrato = ultimo_extrato + timedelta(days = -1)
                ultimo_extrato

            df1 = extrato.posicao_extrato(sc,ultimo_extrato)
            df1['data_ultimo_extrato']=ultimo_extrato          
            df = pd.concat([df,df1])

#Mapeando as informações da base GPS e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Carteira GPS
carteira_gps = df
carteira_gps=pd.merge(left=carteira_gps,right=produtos_gps,on='IdMysqlProduto',how='left')

In [22]:
carteira_gps.columns

Index(['Data_Posicao', 'NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto',
       'CodigoProduto', 'ISIN', 'quantidade', 'PU', 'SaldoNaData', 'Moeda',
       'classe_sistema', 'subclasse_sistema', 'Esteira', 'Politica', 'Officer',
       'Controller'],
      dtype='object')

In [9]:
#Carteira GPS
carteira_gps = df
carteira_gps=pd.merge(left=carteira_gps,right=produtos_gps,on='IdMysqlProduto',how='left')

In [10]:
#Fazer a conciliação de contas
conciliacao = df[['NomeContaCrm','NomeSupercarteiraCrm']].drop_duplicates()
conciliacao=conciliacao['NomeSupercarteiraCrm'].str.lower()
todas_sc = alocacao_por_sc[['new_name','new_conta_supercarteiraidname']].drop_duplicates()
conciliacao = pd.merge(left=conciliacao, right=todas_sc,left_on='NomeSupercarteiraCrm',right_on='new_name',how='right')
conciliacao = conciliacao[conciliacao['NomeSupercarteiraCrm'].isnull()]
conciliacao.to_excel('conciliacao.xlsx')

In [11]:
#Ajustes nas - futuratmente tirar colunas inúteis
carteira_gps['Moeda']='NA'
carteira_gps = carteira_gps[['DataPosicao','NomeSupercarteiraCrm','NomeContaCrm','NomeDoProduto','CodigoProduto_x','ISIN','QuantidadeCotas','ValorCota','SaldoNaData','Moeda','Classe', 'Subclasse']].rename(columns={'DataPosicao':'Data_Posicao','NomeContaCrm':'Nome_Cliente','NomeDoProduto':'Nome_produto','QuantidadeCotas':'quantidade','ValorCota':'PU','CodigoProduto_x':'CodigoProduto'})
carteira_gps['Esteira']='ex-Gps'
carteira_gps['Nome_produto']=carteira_gps['Nome_produto'].str.lower().apply(lambda x: arruma_espaco(x))


In [12]:
#Filtra os ativos que são imobiliarios e gera uma planilha para consultar na bbg o que é Reit
ativos_imob = carteira_gps[carteira_gps['Classe']=='Imobiliário'][['Nome_produto','ISIN']]
ativos_imob.to_excel('I:/Shared/SAO_Investimentos/Portfolio Management/Tamara/verificar_reits.xlsx')

#Traz a planilha com as infos da bbg
classificacao_imob = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/Base Real Estate.xlsx')

#Separando os Reits dos Iliquidos
carteira_gps = pd.merge(left=carteira_gps,right=classificacao_imob,on='ISIN',how='left').drop(columns=['Nome_produto_y']).rename(columns={'Tipo':'Classificacao_Real_Estate'})
carteira_gps['Classe'].loc[carteira_gps['Classe']=='Imobiliário'] = carteira_gps['Classificacao_Real_Estate'].loc[carteira_gps['Classe']=='Imobiliário']
carteira_gps = carteira_gps.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto_x', 'ISIN', 'quantidade','PU']).sort_values(by=['Esteira'])

#Ajuste das classes (padronização dos nomes das classes) - tirar pontos nas classes("-")
carteira_gps['Classe']=carteira_gps['Classe'].astype('str')
classes = carteira_gps['Classe']
carteira_gps['Classe']= [x.replace(' -','') for x in classes]

#DEixando apenas as colunas necessárias, renomeando as mesmas com nomes mais intuitivos
carteira_gps = carteira_gps[['Data_Posicao','NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto_x','CodigoProduto', 'ISIN', 'quantidade','PU','SaldoNaData', 'Moeda', 'Classe', 'Subclasse', 'Esteira']].rename(columns={'Classe':'classe_sistema', 'Subclasse':'subclasse_sistema','Nome_produto_x':'Nome_produto'})
carteira_gps = pd.merge(left = carteira_gps, right = po_cadastro,right_on ='NomeContaCRM',left_on = 'Nome_Cliente',how='left')
carteira_gps = pd.merge(left = carteira_gps, right = alocacao_por_sc,right_on ='new_conta_supercarteiraidname',left_on = 'Nome_Cliente',how='left')
carteira_gps=carteira_gps[['Data_Posicao','NomeSupercarteiraCrm', 'Nome_Cliente', 'Nome_produto','CodigoProduto', 'ISIN', 'quantidade',
        'PU','SaldoNaData', 'Moeda', 'classe_sistema', 'subclasse_sistema', 'Esteira',
        'new_politicadeinvestimentoidname','Officer', 'Controller']].rename(columns={'new_politicadeinvestimentoidname':'Politica'})
carteira_gps = carteira_gps.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'ISIN', 'quantidade','PU']).sort_values(by=['Esteira'])
carteira_gps.to_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/8 - Carteiras Off/carteira_off_gps.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
#Cria o dataframe de movimentos, retira os produtos que não possuem classe cadastrada e faz depara com a classes de asset allocation.
movimentos_gps = carteira_gps[carteira_gps['classe_sistema']!='nan'].drop_duplicates()
movimentos_gps['classe_sistema']=movimentos_gps['classe_sistema'].apply(lambda x:dicio_classificacoes[x])

#Identifica quais produtos são fundos gestão JBFO - 
movimentos_gps=pd.merge(left=movimentos_gps,right=fundos_jbfo,on='CodigoProduto',how='left')
movimentos_gps['Classificação'] = movimentos_gps['Classificação'].astype('str')
movimentos_gps['classe_sistema']=movimentos_gps.apply(lambda row: row['classe_sistema'] if row['Classificação']=='nan' else row['Classificação'],axis=1 )

In [14]:
#Encontrar a nova tapi (indicada para cada cliente de acordo com a tapi anterior) - retira as duplicidades.
movimentos_gps = pd.merge(left=movimentos_gps,right=mandatos,left_on='Politica',right_on='Anterior',how='left')
movimentos_gps=movimentos_gps.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','PU'])
movimentos_gps = movimentos_gps.dropna(subset=['Novo'])

In [15]:
#Verifica se os clientes possuem H3 Strategy caso sim, eles precisam ter o mandato que some Alt LS + Alt. Crédito + Alt. Multiestrategia (MS)
movimentos_gps['Possui_H3']= movimentos_gps.apply(lambda row: 1 if row['classe_sistema']=='Alternativos Macro Arbitragem - MS - JBFO' else 0,axis=1)

#Calcula o percentual do produto dentro da carteira, o financeiro total e desconsiderando os ativos iliquidos.
percentual_produto=[]
financeiro_total_spe =[]
financeiro_total =[]
mandato = []

for ind,row in movimentos_gps.iterrows():   
        h3=0
        #Soma de todos os ativos
        soma = movimentos_gps[movimentos_gps['NomeSupercarteiraCrm']==row['NomeSupercarteiraCrm']]['SaldoNaData'].sum()
        financeiro_total.append(soma)
        #Soma dos ativos exceto iliquidos
        iliquidos = movimentos_gps[(movimentos_gps['NomeSupercarteiraCrm']==row['NomeSupercarteiraCrm'])&(movimentos_gps['classe_sistema']=='Iliq. P. Equity')]['SaldoNaData'].sum()
        financeiro_total_spe.append(soma-iliquidos)
        #Percentual de cada produto (do financeiro sem contar com os ativos iliquidos)    
        percentual= row['SaldoNaData']/(soma-iliquidos)
        percentual_produto.append(percentual)
        #Somatorio das classes Alt LS + Alt. Crédito + Alt. Multiestrategia para clientes que possuem H3 Strategy.
        h3 = movimentos_gps[(movimentos_gps['Nome_Cliente']==row['Nome_Cliente'])&(row['Novo'][-2:]!='MS')]['Possui_H3'].sum()
        if h3>0:
            mandato_fundo = row['Novo']+' MS'
        else:
            mandato_fundo = row['Novo']
        mandato.append(mandato_fundo)        
    
#Incluir no dataframe as informações geradas acima
movimentos_gps['percentual_produto']=percentual_produto
movimentos_gps['Financeiro_Total']=financeiro_total
movimentos_gps['Financeiro_Total_s/pe']=financeiro_total_spe
movimentos_gps['Portfolio']=mandato                                            
movimentos_gps = movimentos_gps.sort_values(by=['classe_sistema'])
movimentos_gps = movimentos_gps.fillna(0)
movimentos_gps=movimentos_gps.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','SaldoNaData']).drop(columns=['Possui_H3'])

#Deixando apenas as colunas que interessam para o groupby - Agrupando por classe de ativo para calculo de movimentação para chegar no ponto atual dos novos mandatos
movimentos_gps = movimentos_gps.groupby(['Data_Posicao','Nome_Cliente','NomeSupercarteiraCrm','classe_sistema','Anterior','Portfolio','Financeiro_Total','Financeiro_Total_s/pe']).sum().reset_index()

<ipython-input-15-d86ed6b04b8d>:19: RuntimeWarning: divide by zero encountered in double_scalars
  percentual= row['SaldoNaData']/(soma-iliquidos)
<ipython-input-15-d86ed6b04b8d>:19: RuntimeWarning: invalid value encountered in double_scalars
  percentual= row['SaldoNaData']/(soma-iliquidos)


In [16]:
#Traz todos os clientes que deverão ser movimentados
clientes = movimentos_gps['NomeSupercarteiraCrm']
classes = novos_mandatos['Classe'].drop_duplicates()
#clientes = ['@5AGL1']

#Traz as informações por cliente (nomes, valores da posição com e sem iliquidos,o percentual atual e proposto de cada classe nos novos portfolios)
dataframe = []    
for classe in classes:
    lista = []
    for cliente in clientes:  
        movimentos_gps_selecao=movimentos_gps[movimentos_gps['NomeSupercarteiraCrm']==cliente]
        Data_posicao = movimentos_gps_selecao.iloc[0,0]
        Nome_Cliente = movimentos_gps_selecao.iloc[0,1]
        NomeSupercarteiraCrm = cliente
        classe_sistema = classe
        #Retorna o mandato anterior
        Anterior = movimentos_gps_selecao.iloc[0,4]
        #Retorna o mandato atual
        Portfolio = movimentos_gps_selecao.iloc[0,5]
        Financeiro_Total = movimentos_gps_selecao.iloc[0,6]
        Financeiro_Total_pe = movimentos_gps_selecao.iloc[0,7]
        #Traz o saldo de cada cliente por classe
        try:
            Saldo = movimentos_gps_selecao[movimentos_gps_selecao['classe_sistema']==classe_sistema].iat[0,10]
        except:
            Saldo = 0
        try:
            percentual_produto = movimentos_gps_selecao[movimentos_gps_selecao['classe_sistema']==classe_sistema].iat[0,11]
        except:
            percentual_produto = 0
        #Traz o percentual proposto no novo mandato para cada classe    
        percentual_proposto= novos_mandatos[(novos_mandatos['Classe']==classe)& (novos_mandatos['Portfolio']==Portfolio)].iat[0,2]
        
        #Preenche o dataframe as informações acima (formato dicionario)
        dataframe.append({'Data_Posicao':Data_posicao,'Nome_Cliente': Nome_Cliente,'NomeSupercarteiraCrm':NomeSupercarteiraCrm, 'classe_sistema': classe_sistema,'PI_Anterior':Anterior,'PI_Proposta':Portfolio,'Financeiro_Total':Financeiro_Total,'Financeiro_s/PE':Financeiro_Total_pe,'Financeiro_Produto':Saldo,'Posicao_Atual':percentual_produto,'Posicao_Proposta':percentual_proposto})  

#Gera um novo dataframe com os dados tratados anterioremente mais os percentuais atuais e os percentuais recomendados nos novos portfolios.       
movimentos_gps = pd.DataFrame.from_dict(dataframe)
movimentos_gps['Movimento']=(movimentos_gps['Posicao_Proposta']-movimentos_gps['Posicao_Atual'])*movimentos_gps['Financeiro_s/PE']            
movimentos_gps=movimentos_gps.fillna(0)


##Deixa em pivot table os dados para calculo dos valores financeiros das movimentações
#Fazendo um pivot table e configurando o mesmo.
movimentos_gps = pd.pivot_table(data=movimentos_gps,values = ['Posicao_Atual','Posicao_Proposta','Movimento'],columns=['classe_sistema'],index=['Data_Posicao','Nome_Cliente','NomeSupercarteiraCrm','PI_Anterior','PI_Proposta','Financeiro_Total','Financeiro_s/PE'])
movimentos_gps = movimentos_gps.reset_index()
movimentos_gps.columns = movimentos_gps.columns.droplevel(0)

#Configurando as colunas para segregar o que é o percentual atual para o proposto
inicio = ['Data_Posicao','SC_Gestão','SC_OFF','Mandato_Anterior','Mandato_Proposto','Financeiro_Total','Financeiro_s/PE']
classe_atual =list(movimentos_gps.columns[-20:])
classes_proposta=list([str(i)+"-Proposto" for i in classes])
classes_movimento=list([str(i)+"-movimento" for i in  classes])
movimentos_gps.columns = inicio + classes_movimento+classe_atual+classes_proposta

#Soma a classe de Alt.LS Asia com Alt.LS
movimentos_gps['Alternativos Long Short']=movimentos_gps['Alternativos Long Short']+movimentos_gps['Alternativos Long Short Asia - JBFO']


In [17]:
##Gerando os valores financeiros de cada classe.
classes_jbfo = fundos_jbfo['Classificação'].drop_duplicates().to_list()
classes_jbfo.remove('Alternativos Macro Arbitragem - MS - JBFO')
classes_jbfo.remove('Alternativos Long Short Asia - JBFO')

##Segregando o que é veiculo JBFO (Tudo o que está JBFO é o saldo da classe em produtos geridos por nós) e calculando os valores a aplicar/receber em cada classe
#classes_jbfo : Retorna os valores dos veiculos de nossa gestão por classe
#valor_veiculo : o que movimentaremos no veiculo, aplicação sempre prioriza fundos da casa, em caso de resgate prioriza outros produtos.

for classe in classes_jbfo:
    lista = []
    lista2 = []
    for ind,row in movimentos_gps.iterrows():
        cliente = row['SC_OFF']
        posicao_classe = row[classe[:-7]]
        posicao_esperada = row[classe[:-7]+'-Proposto']
        posicao_veiculo =  row[classe]
        posicao_classe = posicao_classe + posicao_veiculo
        movimento = posicao_esperada -  posicao_classe
        financeiro = row['Financeiro_s/PE']
        if  movimento>0:
            valor_veiculo = movimento*financeiro
            valor_outros = 0
        elif (movimento < 0) and (posicao_classe*0.95*-1)>movimento:
            valor_veiculo = posicao_veiculo*financeiro*-1
            valor_outros = (posicao_classe-posicao_veiculo)*financeiro*-1
        elif (movimento <0) and (posicao_classe-posicao_veiculo)>(movimento*-1):
            valor_veiculo = 0
            valor_outros = movimento*financeiro
        elif (movimento <0) and ((posicao_classe-posicao_veiculo)==0):
            valor_veiculo = movimento*financeiro
            valor_outros = 0            
        elif row['Alternativos Long Short Asia - JBFO']>0:
            valor_veiculo = (posicao_esperada -posicao_classe)*financeiro
            valor_outros = 0    
        else:
            valor_veiculo = (posicao_esperada-posicao_classe+(posicao_classe-posicao_veiculo))*financeiro
            valor_outros = (movimento*financeiro)- valor_veiculo         
        lista.append(valor_veiculo)
        lista2.append(valor_outros)

    movimentos_gps[classe+'-movimento'] = lista
    movimentos_gps[classe[:-7]+'-movimento'] = lista2
    
# Unifica o veiculo JBFO de Reits aos outros ativos de mesma classe      
movimentos_gps['Real Estate-movimento'] = movimentos_gps['Real Estate-movimento']+ movimentos_gps['Real Estate - JBFO-movimento'] 

In [18]:
# #Localiza os emails dos responsaveis por cada sc e retira da lista os clientes que estão distrato

movimentos_gps = pd.merge(left=movimentos_gps,right=responsaveis,left_on='SC_OFF',right_on='SC/ContaMov',how='left')
movimentos_gps = pd.merge(left=movimentos_gps,right=distrato,left_on='SC_OFF',right_on='new_name',how='left')
movimentos_gps = movimentos_gps[movimentos_gps['new_contratoname']!='Distrato em Andamento']
movimentos_gps = movimentos_gps.drop(columns=['Conta','SC/ContaMov','new_name','new_contrato','new_contratoname']).drop_duplicates()

#Substitui os campos controllers, dpms que estão em branco para o PM off.
movimentos_gps['RM'] = movimentos_gps['RM'].fillna('tamara alves')
movimentos_gps['Co-RM'] = movimentos_gps['Co-RM'].fillna('tamara alves')
movimentos_gps['Controller'] = movimentos_gps['Controller'].fillna('tamara alves')
movimentos_gps['Controller Backup'] = movimentos_gps['Controller Backup'].fillna('tamara alves')

#Retorna os emails dos responsáveis por cada carteira
emails_envio = []
emails_usuarios['fullname']=emails_usuarios['fullname'].str.lower()
for idx, row in movimentos_gps.iterrows(): 
    lista_email=[]
    if row['RM'] in emails_usuarios['fullname'].to_list():  
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']== row['RM']].iat[0,2])
    if row['Co-RM'] in emails_usuarios['fullname'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']==row['Co-RM']].iat[0,2])
    if row['Controller'] in emails_usuarios['fullname'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']==row['Controller']].iat[0,2])
    if row['Controller Backup'] in emails_usuarios['fullname'].to_list():
        lista_email.append(emails_usuarios.loc[emails_usuarios['fullname']==row['Controller Backup']].iat[0,2]) 
    emails_envio.append(lista_email)
movimentos_gps['emails_aprovadores']= emails_envio        

In [19]:
#Retira das colunas os nomes 'movimento' e 'proposto' pois não são necessários para o envio do e-mail de movimentação)
colunas = movimentos_gps.columns
colunas = [col.replace('-movimento','') for col in colunas]
colunas = [col.replace('-Proposto','') for col in colunas]
movimentos_gps.columns = colunas

#Retira as classes que não conseguimos movimentar por questões diversas (tamanho, etc...)
movimentos_gps = movimentos_gps.drop(columns={'Iliq. P. Equity','Alternativos Long Short Asia','Alternativos Long Short Asia - JBFO','Real Estate - JBFO'})

In [21]:
officers = movimentos_gps['RM'].drop_duplicates().to_list()
#Teste = officers = ['bruna casanova']
classes_pm = list(movimentos_gps.columns[7:23])


##Gera o email que será enviado para cada officer

#Puxa os emails dos destinatários responsáveis pela carteira.
for officer in officers:
    df = movimentos_gps[movimentos_gps['RM']==officer]
    clientes =df['SC_Gestão']
    sc_off = df['SC_OFF']
    txt_email=[]
    emails = [x for i,elem in df['emails_aprovadores'].items() for x in elem]
    emails=list(set(emails))
    emails = [x for x in emails if x is not None]
    
    txt_inicio = f'''Prezados,<br>   
        Seguem as movimentações propostas para adequação dos portfólios atuais para os novos mandatos. Caso tenham alguma sugestão vinda da equipe dos DPMs, favor desconsiderar.<br><br>    
        Os portfólios que possuem "MS", são aqueles que possuem o H3 Composite (classe Multiestratégia)-<br><br>
        ''' 
#Gera os movimentos que cada sc precisa fazer.
    for sc in sc_off:
        
        df = movimentos_gps[movimentos_gps['RM']==officer]
        df = df[df['SC_OFF']==sc]
        df[df['SC_OFF']==sc][['SC_Gestão']].iloc[0,0]
        if df.empty:
            txt = txt1 = f'<ul><li>Sem movimentações a fazer.</li>'
        else:    
            mandato_atual = df['Mandato_Anterior'].iloc[0]
            mandato_proposto = df['Mandato_Proposto'].iloc[0]
            patrimonio_total = formatar(df['Financeiro_Total'].iloc[0])
            patrimonio_s_pe = formatar(df['Financeiro_s/PE'].iloc[0]) 
            df_financeiro= df.iloc[:, 0:23]
            df_atual= df.iloc[:, 23:39]
            df_proposto=df.iloc[:, 39:55]   
            SC_On = df['SC_Gestão'].iloc[0]
            data_posicao=df.iat[0,0]
            data_posicao = "{}/{}/{}".format(data_posicao.day,data_posicao.month,data_posicao.year)
            txt = f'''<b>Cliente {sc} / {SC_On}:</b><br>
            *Data Posição : {data_posicao},<br>
            * Mandato : {mandato_atual},<br>
            * Mandato Proposto : {mandato_proposto},<br>
            * PL Total : {patrimonio_total},<br>
            * PL desconsiderando Ilíquidos e Imobiliarios iliquidos : {patrimonio_s_pe},<br>'''
            for classe_pm in classes_pm:     
                if classe_pm == 'Alternativos Macro Arbitragem - MS - JBFO':
                    classe_pm_aloc = 'Alternativos Macro Arbitragem - MS - JBFO'
                    ativo = 'H3 Composisite'
                elif classe_pm[-7:]==' - JBFO':
                    classe_pm_aloc = classe_pm[:-7]
                    ativo = dicio_veiculos[classe_pm]
                else:    
                    classe_pm_aloc=classe_pm
                    ativo = dicio_veiculos[classe_pm]            
                percentual_proposto_valor=df_proposto[classe_pm_aloc].iloc[0]*100
                try:
                    percentual_atual_valor=(df_atual[classe_pm_aloc].iloc[0]+df_atual[classe_pm_aloc+' - JBFO'].iloc[0])*100
                except:
                    percentual_atual_valor=df_atual[classe_pm_aloc].iloc[0]*100


                #                 percentual = novos_mandatos[(novos_mandatos['Classe']==classe)&(novos_mandatos['Portfolio']==mandato_proposto)].iat[0,2]*100
                #             else:
                #                 classe=classe_pm
                #                 percentual = novos_mandatos[(novos_mandatos['Classe']==classe)&(novos_mandatos['Portfolio']==mandato_proposto)].iat[0,2]*100
                try:
                    valor = df_financeiro[classe_pm].iloc[0]
                    valor = round(valor,-4)
                    classe = classe_pm
                except:
                    valor=0            
                percentual_atual_valor ='{0:.2f}%'.format(percentual_atual_valor)
                percentual_proposto_valor ='{0:.2f}%'.format(percentual_proposto_valor)
                if valor < 0:
                    valor = abs(valor) 
                    valor=formatar(valor)
                    txt1 = f'<ul><li>Resgate de ${valor} de {ativo} para levar a alocação da classe de {percentual_atual_valor} para o ponto tático de {percentual_proposto_valor}.</li>'
                elif valor > 0:
                    valor = abs(valor)
                    valor=formatar(valor)
                    txt1 = f'<ul><li>Aplicação de ${valor} de {ativo} para levar a alocação da classe de {percentual_atual_valor} para o ponto tático de {percentual_proposto_valor}.</li>'
                else:
                    txt1=''
                txt += txt1 + '</ul>'

            txt_email.append(txt)
        corpo = f"{txt_inicio}{'<br>'.join(txt_email)}"

    subject= f'Movimentos offshore - Novos Mandatos - Base ex-GPS'
    to=['tamara.alves@jbfo.com','portfolio@jbfo.com']+emails
    email = Email(to = to , subject = subject, text= corpo,send = True)    
    
#Gera o excel de cada officer.
    df = df.reset_index(drop=True)
    df= df.iloc[:, 0:23]
    df.to_excel(f'O:/SAO/\CICH_All\/Portfolio Management/20 - Offshore/1 - Movimentao_officer/{officer}.xlsx')
    






[Emailer]: Sending mail "Movimentos offshore - Novos Mandatos - Base ex-GPS" to ['tamara.alves@jbfo.com', 'portfolio@jbfo.com', 'tamara.alves@juliusbaer.com', 'carina.santos@juliusbaer.com', 'patricia.bianchi@juliusbaer.com']
[Emailer]: Sending mail "Movimentos offshore - Novos Mandatos - Base ex-GPS" to ['tamara.alves@jbfo.com', 'portfolio@jbfo.com', 'tamara.alves@juliusbaer.com', 'stefano.spada@juliusbaer.com']
[Emailer]: Sending mail "Movimentos offshore - Novos Mandatos - Base ex-GPS" to ['tamara.alves@jbfo.com', 'portfolio@jbfo.com', 'roberta.botelho@juliusbaer.com', 'tamara.alves@juliusbaer.com', 'danilo.reis@juliusbaer.com', 'otavio.botrel@juliusbaer.com']
[Emailer]: Sending mail "Movimentos offshore - Novos Mandatos - Base ex-GPS" to ['tamara.alves@jbfo.com', 'portfolio@jbfo.com', 'vanessa.sato@juliusbaer.com', 'bruna.casanova@juliusbaer.com', 'bruna.almeida@juliusbaer.com', 'bruno.rebechi@juliusbaer.com', 'tamara.alves@juliusbaer.com', 'luciana.braga@juliusbaer.com']
[Emailer]

In [ ]:
#Pegando o primeiro dia de cada mes
data = fdt.hoje()
data = data - relativedelta(days=30)
data = date(data.year, data.month, 1) 
data=pd.to_datetime(data)

#Carteira do ultimo extrato
alocacao = rel_off_c.carteira_por_peso(data)
alocacao['NomeClasse']=alocacao['NomeClasse'].str.lower()

#dicionario de classes - Deixando tudo minusculo e equiparando com as classes de asset allocation
dicio_classes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classes=dicio_classes[['Classe_Alocacao_sistema','Classe GPS']]
dicio_classes = dict(dicio_classes.values)

del dicio_classes[np.nan]
dicio_classes = dict((k.lower(), v.lower()) for k, v in dicio_classes.items()) 
alocacao['NomeClasse']=alocacao['NomeClasse'].apply(lambda x:dicio_classes[x] )
alocacao=alocacao.dropna(subset=['nome_reduzido'])


del dicio_classificacoes[np.nan]
dicio_classificacoes = dict((k.lower(), v.lower()) for k, v in dicio_classificacoes.items()) 
alocacao['NomeClasse']=alocacao['NomeClasse'].apply(lambda x:dicio_classificacoes[x])

#Fazendo o merge com todos os possíveis portfolios
#novos_mandatos['Classe'] = novos_mandatos['Classe'].str.lower()
novos_mandatos = novos_mandatos[novos_mandatos['Classe']!='alternativos long short asia']
novos_mandatos = novos_mandatos[novos_mandatos['Portfolio'].str.contains('MS') == False]

#Puxar os clientes para o for
clientes = alocacao['nome_reduzido'].drop_duplicates().to_list()
clientes = [x for x in clientes if x is not None]

#Puxar os portfolios para o for
portfolios = novos_mandatos['Portfolio'].drop_duplicates().to_list()

#dicionario de classes - Deixando tudo minusculo e equiparando com as classes de asset allocation
dicio_classes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classes=dicio_classes[['Classe_Alocacao_sistema','Classe GPS']]
dicio_classes = dict(dicio_classes.values)

#Rentabilidade das carteiras
rentabilidade = rel_off_c.rentabilidade(data)
rentabilidade = rentabilidade.loc[rentabilidade.groupby('nome_reduzido').DataProcessamentoFim.idxmin()].reset_index(drop=True)
rentabilidade = rentabilidade[['nome_reduzido','PontosRisco','RetornoLiquido_NoAno','RetornoLiquido_12Meses']]

In [ ]:
dicio_classes

In [ ]:
pos = pd.DataFrame()
tracking= []
clientes = ['REPAR_JB']

for cliente in clientes:
    nome=cliente
    df = alocacao[alocacao['nome_reduzido']==nome]
    df = df[df['DataProcessamentoFim'] == df['DataProcessamentoFim'].max()]
    df = df.groupby(['nome_reduzido','NomeClasse'])[['PcInicioAno']].sum().reset_index()
    
    df= pd.merge(left=df,right=novos_mandatos,left_on='NomeClasse',right_on='Classe',how='right')
    df = df[df['Classe'].str.contains('jbfo') == False]
    df['nome_reduzido']=df['nome_reduzido'].fillna(nome)
    df['PcInicioAno']= df['PcInicioAno'].fillna(0)
    df = df.rename(columns={'PcInicioAno':'Atual'})    
    df= df[['Portfolio','Classe','Tático','Atual']]
    
#     for portfolio in portfolios:
#         analise = df[df['Portfolio']==portfolio]
#         analise = analise[['Classe','Tático','Atual']]
#         te=calc.tracking_error(analise)
#         tracking.append({'Cliente': nome, 'portfolio': portfolio,'TE': te})
        
    


In [ ]:
#port_recomendado.to_excel('mandato_proposto.xlsx')
calc.vols